In [1]:
!nvidia-smi

Fri Jun 17 16:38:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 472.19       Driver Version: 472.19       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   51C    P8    12W /  N/A |    337MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import dgl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader
from dgl.dataloading import GraphDataLoader
from dgl.nn import GraphConv,SAGEConv,MaxPooling
import dgl.data
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PPIDataset
from dgl import DropEdge,AddSelfLoop,GCNNorm,LaplacianPE
from torch_geometric.nn import GCNConv, PairNorm
from torch_geometric.utils import dropout_adj
from functools import namedtuple
from load_graph import simple_dataloader, Load_graph
print(f"torch.__version__: {torch.__version__}")
print(f"dgl.__version__: {dgl.__version__}")

torch.__version__: 1.11.0+cu113
dgl.__version__: 0.8.2


In [127]:
def train(g, model, epochs, lr):
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.8, patience=3,threshold=0.0001, verbose=True, min_lr=1e-8)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(epochs):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
        if scheduler is not None:
            scheduler.step(val_acc)

In [3]:
dataset = Load_graph('cora', 'node')
# print('Number of categories:', dataset.num_classes)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [4]:
g = dataset[0].to('cuda')
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'train_mask': tensor([False, False, False,  ..., False, False, False], device='cuda:0'), 'label': tensor([4, 4, 4,  ..., 4, 3, 3], device='cuda:0'), 'val_mask': tensor([False, False,  True,  ..., False, False, False], device='cuda:0'), 'test_mask': tensor([ True,  True, False,  ..., False, False, False], device='cuda:0'), 'feat': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0526, 0.0000]],
       device='cuda:0')}
Edge features
{'__orig__': tensor([  298,  9199,  1153,  ..., 10415,  5255,  6356], device='cuda:0')}


In [128]:
class GCNBlock(nn.Module):
    def __init__(self, in_feats, out_feats, activation=None, pair_norm=False, add_self_loops=False, drop_edge=0):
        super(GCNBlock, self).__init__()
#         self.conv = SAGEConv(in_feats, out_feats,'mean')
        self.conv = GraphConv(in_feats, out_feats)
        self.dropedge = DropEdge(p=drop_edge)
        self.self_loops = AddSelfLoop()
        self.norm = PairNorm()
        self.activation = activation
        self.pair_norm = pair_norm
        self.add_self_loops = add_self_loops
        self.drop_edge = drop_edge

    def forward(self, g, in_feat):
        if self.drop_edge:
            g = self.dropedge(g)
            self.add_self_loops = True
        if self.add_self_loops:
            g = self.self_loops(g)
        h = self.conv(g, in_feat)
        if self.pair_norm:
            h = self.norm(h)
        if self.activation:
            h = self.activation(h)
        return h

class GCN(nn.Module):
    def __init__(self, in_feats, n_hidden, n_classes, n_layers, activation, dropout=0., 
                 pair_norm=False, add_self_loops=False, drop_edge=0):
        super(GCN, self).__init__()
        if dropout:
            self.dropout = nn.Dropout(p=dropout)
        else:
            self.dropout = 0.
        self.layers = nn.ModuleList()
        # input layer
        self.layer1 = GCNBlock(in_feats, n_hidden, activation, pair_norm=pair_norm,add_self_loops=add_self_loops, drop_edge=drop_edge)

        self.layers = nn.ModuleList()
        # hidden layers
        for i in range(n_layers - 1):
            self.layers.append(GCNBlock(n_hidden, n_hidden, activation, pair_norm=pair_norm,add_self_loops=add_self_loops, drop_edge=drop_edge))
        # output layer
        self.layers.append(GCNBlock(n_hidden, n_classes, activation=None, pair_norm=pair_norm,add_self_loops=add_self_loops,drop_edge=drop_edge))
        # self.classify = nn.Linear(n_classes, n_classes)

    def forward(self, g, features):
        h = self.layer1(g, features)
        for idx, layer in enumerate(self.layers):
            if idx > 0 and self.dropout:
                h = self.dropout(h)
            h = layer(g, h)
        if self.dropout:
            h = self.dropout(h)
        return h

In [157]:
g = dataset[0].to('cuda')
model = GCN(g.ndata['feat'].shape[1], 512, dataset.num_classes, 
            n_layers=1,
            activation=F.relu, 
            dropout=0, 
            pair_norm=False, 
            add_self_loops=True, 
            drop_edge=0).to('cuda')
train(g, model, epochs=50, lr=0.03)

In epoch 0, loss: 1.946, val acc: 0.246 (best 0.246), test acc: 0.244 (best 0.244)
In epoch 5, loss: 0.544, val acc: 0.776 (best 0.778), test acc: 0.794 (best 0.789)
In epoch 10, loss: 0.031, val acc: 0.788 (best 0.796), test acc: 0.817 (best 0.828)
Epoch 00012: reducing learning rate of group 0 to 2.4000e-02.
In epoch 15, loss: 0.003, val acc: 0.784 (best 0.796), test acc: 0.799 (best 0.828)
Epoch 00016: reducing learning rate of group 0 to 1.9200e-02.
Epoch 00020: reducing learning rate of group 0 to 1.5360e-02.
In epoch 20, loss: 0.001, val acc: 0.776 (best 0.796), test acc: 0.797 (best 0.828)
Epoch 00024: reducing learning rate of group 0 to 1.2288e-02.
In epoch 25, loss: 0.000, val acc: 0.778 (best 0.796), test acc: 0.793 (best 0.828)
Epoch 00028: reducing learning rate of group 0 to 9.8304e-03.
In epoch 30, loss: 0.000, val acc: 0.778 (best 0.796), test acc: 0.792 (best 0.828)
Epoch 00032: reducing learning rate of group 0 to 7.8643e-03.
In epoch 35, loss: 0.000, val acc: 0.778 (

ppi train

In [125]:
from sklearn.metrics import f1_score,accuracy_score

def micro_f1_score(logits, labels):
    predict = np.where(logits.detach().numpy() >= 0., 1, 0)
#     print(labels.numpy(),predict)
    return f1_score(labels.numpy(), predict, average='micro')


def evaluate(model, g, feats, labels):
    with torch.no_grad():
        model.eval()
        logits = model(g, feats)
    return micro_f1_score(logits.cpu() , labels.cpu() )

def train(epochs, lr, n_hidden, n_layers, activation, dropout=0., pair_norm=False, add_self_loops=False, drop_edge=0):
    train_dataset = PPIDataset(mode='train')
    valid_dataset = PPIDataset(mode='valid')
    test_dataset = PPIDataset(mode='test')

    # data loader
    train_loader = GraphDataLoader(train_dataset, batch_size=1, shuffle=True)
    valid_loader = GraphDataLoader(valid_dataset, batch_size=1, shuffle=False)
    test_loader = GraphDataLoader(test_dataset, batch_size=1, shuffle=False)

    g = train_dataset[0].to('cuda')
    n_classes = train_dataset.num_labels
    n_features = g.ndata['feat'].shape[1]

    model = GCN(n_features, 
                n_hidden, 
                n_classes,
                n_layers=n_layers,
                activation=activation, 
                dropout=dropout, 
                pair_norm=pair_norm,
                add_self_loops=add_self_loops,
                drop_edge=drop_edge
               ).to('cuda')
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.8, patience=5,threshold=0.0001, verbose=True, min_lr=1e-8)
    best_val_f1 = 0
    best_test_f1 = 0
    for e in range(epochs):
        losses = []
        train_scores = []
        for bg in train_loader:
            bg = bg.to('cuda')
#             transform1 = AddSelfLoop()
#             bg = transform1(bg)
            logits = model(bg, bg.ndata['feat'])
            labels = bg.ndata['label'].float()
            loss = F.binary_cross_entropy_with_logits(logits, labels)
            losses.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        val_scores = [evaluate(model, bg.to('cuda'), bg.to('cuda').ndata['feat'], bg.to('cuda').ndata['label']) for bg in valid_loader]
        test_scores = [evaluate(model, bg.to('cuda'), bg.to('cuda').ndata['feat'], bg.to('cuda').ndata['label']) for bg in test_loader]
        
        val_f1 = np.array(val_scores).mean()
        test_f1 = np.array(test_scores).mean()
        
        if best_val_f1 < val_f1:
            best_val_f1 = val_f1
            best_test_f1 = test_f1
            

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val f1: {:.3f} (best {:.3f}), test f1: {:.3f} (best {:.3f})'.format(
                e, np.array(losses).mean(), val_f1, best_val_f1, test_f1, best_test_f1))
        if scheduler is not None:
            scheduler.step(val_f1)

In [1433]:
train(epochs=600,lr=0.005,n_hidden=256,n_layers=5,
      activation=F.tanh, dropout=0., pair_norm=False, 
      add_self_loops=True, drop_edge=0)

In epoch 0, loss: 0.608, val f1: 0.498 (best 0.498), test f1: 0.508 (best 0.508)
In epoch 5, loss: 0.352, val f1: 0.714 (best 0.714), test f1: 0.736 (best 0.736)
In epoch 10, loss: 0.217, val f1: 0.823 (best 0.823), test f1: 0.850 (best 0.850)
In epoch 15, loss: 0.141, val f1: 0.884 (best 0.884), test f1: 0.906 (best 0.906)
In epoch 20, loss: 0.105, val f1: 0.904 (best 0.904), test f1: 0.926 (best 0.926)
In epoch 25, loss: 0.073, val f1: 0.931 (best 0.931), test f1: 0.951 (best 0.951)
In epoch 30, loss: 0.059, val f1: 0.939 (best 0.939), test f1: 0.958 (best 0.958)
In epoch 35, loss: 0.046, val f1: 0.947 (best 0.947), test f1: 0.965 (best 0.965)
In epoch 40, loss: 0.034, val f1: 0.955 (best 0.955), test f1: 0.971 (best 0.971)
In epoch 45, loss: 0.029, val f1: 0.958 (best 0.958), test f1: 0.973 (best 0.973)
In epoch 50, loss: 0.032, val f1: 0.948 (best 0.958), test f1: 0.964 (best 0.973)
Epoch 00052: reducing learning rate of group 0 to 4.0000e-03.
In epoch 55, loss: 0.019, val f1: 0.96

Epoch 00347: reducing learning rate of group 0 to 5.3169e-07.
In epoch 350, loss: 0.004, val f1: 0.973 (best 0.973), test f1: 0.984 (best 0.984)
Epoch 00353: reducing learning rate of group 0 to 4.2535e-07.
In epoch 355, loss: 0.004, val f1: 0.973 (best 0.973), test f1: 0.984 (best 0.984)
Epoch 00359: reducing learning rate of group 0 to 3.4028e-07.
In epoch 360, loss: 0.004, val f1: 0.973 (best 0.973), test f1: 0.984 (best 0.984)
Epoch 00365: reducing learning rate of group 0 to 2.7223e-07.
In epoch 365, loss: 0.004, val f1: 0.973 (best 0.973), test f1: 0.984 (best 0.984)
In epoch 370, loss: 0.004, val f1: 0.973 (best 0.973), test f1: 0.984 (best 0.984)
Epoch 00371: reducing learning rate of group 0 to 2.1778e-07.
In epoch 375, loss: 0.004, val f1: 0.973 (best 0.973), test f1: 0.984 (best 0.984)
Epoch 00377: reducing learning rate of group 0 to 1.7422e-07.
In epoch 380, loss: 0.004, val f1: 0.973 (best 0.973), test f1: 0.984 (best 0.984)
Epoch 00383: reducing learning rate of group 0 